In [1]:
import requests
import json
import numpy as np
import pandas as pd

In [2]:
from config import walkscore_api_key

In [15]:
csv_path = "Project1_AmazonSites.csv"
city_csv = pd.read_csv(csv_path)
# to remove all null values
city_df = city_csv.iloc[:39,:9]
# change zip to only integer
city_df['zip code'] = city_df['zip code'].astype(int)
for index, zip in city_df['zip code'].iteritems():
    if len(str(zip)) == 4:
        city_df['zip code'].iloc[index] = "0"+str(zip)
city_df.head()

C:\Users\hleao\AppData\Local\Continuum\anaconda3\envs\Python\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,amazon city,site,site name,address,city,state,zip code,latitude,longitude
0,Northern Virginia Area,Site1,Center for Innovation Technology,2214 Rock Hill Road,Herndon,VA,20170,38.96,-77.42
1,Northern Virginia Area,Site2,Crystal City and Potomac Yard,3535 S Ball Street,Arlington,VA,22202,38.84,-77.05
2,Washington DC,Site1,Nationals Park/ D.C United Stadium,32-60 R Street,Washington,DC,20024,38.86,-77.01
3,Washington DC,Site2,Union Station,10 G Street,Washington,DC,20002,38.89,-77.00
4,Washington DC,Site3,Hill-East Neighborhood,1430 Pennsylvania Avenue,Washington,DC,20003,38.88,-76.97


In [75]:
city_param = city_df[['latitude', 'longitude', 'address','site name','city','amazon city']]
city_param.head()

,latitude,longitude,address,site name,city,amazon city
0,38.96,-77.42,2214 Rock Hill Road,Center for Innovation Technology,Herndon,Northern Virginia Area
1,38.84,-77.05,3535 S Ball Street,Crystal City and Potomac Yard,Arlington,Northern Virginia Area
2,38.86,-77.01,32-60 R Street,Nationals Park/ D.C United Stadium,Washington,Washington DC
3,38.89,-77.00,10 G Street,Union Station,Washington,Washington DC
4,38.88,-76.97,1430 Pennsylvania Avenue,Hill-East Neighborhood,Washington,Washington DC


In [76]:
base_url = "http://api.walkscore.com/score?"
walkscore_list = []

for city in city_param.values:

    address_decoded = str(city[2]).replace(" ","%20")
    params = f"format=json&address={address_decoded}&lat={city[0]}&lon={city[1]}&transit=1&bike=1&wsapikey={walkscore_api_key}"
    full_url = base_url + params
    walkscore_response = requests.get(full_url)
    walkscore_json = walkscore_response.json()
    
    if 'transit' not in walkscore_json.keys():
        if 'bike' not in walkscore_json.keys():
            walkscore_list.append({'Site Name':city[3],
                     'City':city[4],
                     'Amazon City':city[5],
                     'Description':walkscore_json['description'],
                     'Walk score':walkscore_json['walkscore'],
                     'Transit score':None,
                     'Transit description':None,
                     'Bike score':None,
                     'Bike description':None})
        else:
            walkscore_list.append({'Site Name':city[3],
                     'City':city[4],
                     'Amazon City':city[5],
                     'Description':walkscore_json['description'],
                     'Walk score':walkscore_json['walkscore'],
                     'Transit score':None,
                     'Transit description':None,
                     'Bike score':walkscore_json['bike']['score'],
                     'Bike description':walkscore_json['bike']['description']})
    else:
        try:
            walkscore_list.append({'Site Name':city[3],
                     'City':city[4],
                     'Amazon City':city[5],
                     'Description':walkscore_json['description'],
                     'Walk score':walkscore_json['walkscore'],
                     'Transit score':walkscore_json['transit']['score'],
                     'Transit description':walkscore_json['transit']['description'],
                     'Bike score':walkscore_json['bike']['score'],
                     'Bike description':walkscore_json['bike']['description']})
        except:
            walkscore_list.append({'Site Name':city[3],
                     'City':city[4],
                     'Amazon City':city[5],
                     'Description':walkscore_json['description'],
                     'Walk score':walkscore_json['walkscore'],
                     'Transit score':walkscore_json['transit']['score'],
                     'Transit description':walkscore_json['transit']['description'],
                     'Bike score':None,
                     'Bike description':None})

#Pomona Fairplex and The High Street Site had bike key errors

In [77]:
walkscore_df = pd.DataFrame(walkscore_list)
walkscore_df.head()

,Amazon City,Bike description,Bike score,City,Description,Site Name,Transit description,Transit score,Walk score
0,Northern Virginia Area,None,NaN,Herndon,Car-Dependent,Center for Innovation Technology,None,NaN,43
1,Northern Virginia Area,Very Bikeable,82.0,Arlington,Somewhat Walkable,Crystal City and Potomac Yard,Some Transit,47.0,69
2,Washington DC,Somewhat Bikeable,34.0,Washington,Car-Dependent,Nationals Park/ D.C United Stadium,Good Transit,57.0,22
3,Washington DC,Biker's Paradise,97.0,Washington,Walker's Paradise,Union Station,Excellent Transit,84.0,91
4,Washington DC,Bikeable,58.0,Washington,Car-Dependent,Hill-East Neighborhood,Excellent Transit,71.0,10


In [78]:
walkscore_df = walkscore_df[['Site Name','Amazon City','City','Walk score','Description','Transit score','Transit description','Bike score','Bike description']]
walkscore_df

,Site Name,Amazon City,City,Walk score,Description,Transit score,Transit description,Bike score,Bike description
0,Center for Innovation Technology,Northern Virginia Area,Herndon,43,Car-Dependent,NaN,None,NaN,None
1,Crystal City and Potomac Yard,Northern Virginia Area,Arlington,69,Somewhat Walkable,47.0,Some Transit,82.0,Very Bikeable
2,Nationals Park/ D.C United Stadium,Washington DC,Washington,22,Car-Dependent,57.0,Good Transit,34.0,Somewhat Bikeable
3,Union Station,Washington DC,Washington,91,Walker's Paradise,84.0,Excellent Transit,97.0,Biker's Paradise
4,Hill-East Neighborhood,Washington DC,Washington,10,Car-Dependent,71.0,Excellent Transit,58.0,Bikeable
5,Shaw-Howard University Area,Washington DC,Washington,91,Walker's Paradise,80.0,Excellent Transit,96.0,Biker's Paradise
6,Warner Center West San Fernando Valley,Los Angeles,Woodland Hills,62,Somewhat Walkable,59.0,Good Transit,65.0,Bikeable
7,New Hall Ranch Santa Clarita Valley,Los Angeles,Santa Clarita,67,Somewhat Walkable,NaN,None,NaN,None
8,Pomona Fairplex,Los Angeles,Pomona,17,Car-Dependent,25.0,Some Transit,NaN,None
9,Suffolk Downs,Boston,Boston,64,Somewhat Walkable,60.0,Good Transit,51.0,Bikeable


In [93]:
amazon_city_group = walkscore_df.groupby(['Amazon City'])
amazon_city_group[['Walk score', 'Bike score', 'Transit score']].mean()



,Walk score,Bike score,Transit score
Amazon City,,,
Atlanta,54.250000,74.00,55.00
Austin,29.571429,49.00,39.00
Boston,69.500000,68.50,76.25
Chicago,76.375000,78.25,81.00
Los Angeles,48.666667,65.00,42.00
New York,86.250000,76.00,95.75
Northern Virginia Area,56.000000,82.00,47.00
Raleigh,32.666667,62.00,73.00
Washington DC,53.500000,71.25,73.00
